# dask로 데이터 열고, 전처리하기

In [1]:
import pandas as pd 
import glob
from tqdm import tqdm
import dask.dataframe as dd

def make_sido_code(row):
    return str(row["시군구코드"])[:2]

### (1) 건축물대장 기본개요

In [64]:
%%time

# 건축물대장 기본개요
columns = ["관리건축물대장PK","관리상위건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명", \
            "대지위치","도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명","블록","로트","외필지수", \
            "새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번","새주소부번","지역코드","지구코드","구역코드", \
            "지역코드명","지구코드명","구역코드명","생성일자"]

basis = dd.read_csv("data/rawdata/building/기본개요/mart_djy_01.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
print(f"건축물대장 기본개요의 분할된 개수는 {basis.npartitions}개입니다.")

busan_basis = basis[["관리건축물대장PK", "관리상위건축물대장PK", "시군구코드"]]
busan_basis["시도코드"] = busan_basis.apply(make_sido_code, axis=1, meta=object)
busan_basis = busan_basis[busan_basis["시도코드"] == "26"].drop(["시군구코드"], axis=1)

# preprocessing
busan_basis["관리건축물대장PK"] = busan_basis["관리건축물대장PK"].map(lambda x: x.strip())
busan_basis["관리상위건축물대장PK"] = busan_basis["관리상위건축물대장PK"].map(lambda x: x.strip())
busan_basis = busan_basis.drop("시도코드", axis=1)

# convert dask to pandas
busan_basis = busan_basis.compute()

# save csv
busan_basis.to_csv("data/refined-data/busan-기본개요.csv", encoding="utf-8", index=False)

건축물대장 기본개요의 분할된 개수는 80개입니다.
Wall time: 9min


### (2) 건축물대장 주택가격

In [65]:
%%time

# 건축물대장 주택가격

# names=columns
# columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치","도로명대지위치", \
#             "건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명","블록","로트","외필지수","새주소도로코드", \
#             "새주소지상지하코드","새주소본번","새주소부번","기준일자","주택가격","생성일자"]

price = dd.read_csv("data/rawdata/building/주택가격/mart_djy_08.txt", sep="\|", dtype=str, encoding='cp949',
                        on_bad_lines="skip", header=None, engine='python')

# the number of partitions
print(f"건축물대장의 주택가격이 분할된 파일 개수는 {price.npartitions}개입니다.")

# save partitions
# price.to_csv("data/rawdata/building/주택가격/csvs", encoding="utf-8")

price_df = price[[0,8,23,24]]
price_df.columns = ["관리상위건축물대장PK","시군구코드","주택가격","생성일자"]
price_df["시도코드"] = price_df.apply(make_sido_code, axis=1, meta=object)
price_df = price_df.loc[price_df["시도코드"] == "26"]

# preprocessing
price_df["관리상위건축물대장PK"] = price_df["관리상위건축물대장PK"].map(lambda x: x.strip())
price_df = price_df.map_partitions(lambda df: df.sort_values(by="생성일자", ascending=False))
price_df = price_df.drop(["시도코드"], axis=1).drop_duplicates()

# convert dask to pandas
busan_price_pdf = price_df.compute()

# save csv
busan_price_pdf.to_csv("data/refined-data/busan-price.csv", encoding="utf-8", index=False)

# row 개수
print(f"주택가격의  행수는 {busan_price_pdf.shape[0]}개입니다.")

건축물대장의 주택가격이 분할된 파일 개수는 416개입니다.
주택가격의  행수는 9183677개입니다.
Wall time: 43min 27s


### (3) 건축물대장 표제부

In [102]:
%%time

columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치", \
            "도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명", \
            "블록","로트","외필지수","새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번", \
            "새주소부번","동명","주부속구분코드","주부속구분코드명","대지면적(㎡)","건축면적(㎡)", \
            "건폐율(%)","연면적","용적률산정연면적","용적률","구조코드","구조코드명","기타구조", \
            "주용도코드","주용도코드명","기타용도","지붕코드","지붕코드명","기타지붕","세대수(세대)", \
            "가구수(가구)","높이(m)","지상층수","지하층수","승용승강기수","비상용승강기수","부속건축물수", \
            "부속건축물면적","총동연면적","옥내기계식대수(대)","옥내기계식면적","옥외기계식대수(대)", \
            "옥외기계식면적","옥내자주식대수(대)","옥내자주식면적","옥외자주식대수(대)","옥외자주식면적", \
            "허가일","착공일","사용승인일","허가번호년","허가번호기관코드","허가번호기관코드명", \
            "허가번호구분코드","허가번호구분코드명","호수(호)","에너지효율등급","에너지절감율", \
            "에너지EPI점수","친환경건축물등급","친환경건축물인증점수","지능형건축물등급","지능형건축물인증점수", \
            "생성일자","내진설계적용여부","내진능력"]

building = dd.read_csv("data/rawdata/building/표제부/mart_djy_03.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
print(f"건축물대장 표제부의 분할된 개수는 {building.npartitions}개입니다.")

busan_building = building[["관리건축물대장PK","대장종류코드명","도로명대지위치","시군구코드","시군구코드", \
                            "대지면적(M2)","건축면적(M2)","건폐율(%)", "연면적","주용도코드명","가구수(가구)","높이(M)","지상층수","지하층수", \
                            "사용승인일","허가번호기관코드","친환경건축물등급","친환경건축물인증점수","지능형건축물등급","지능형건축물인증점수","내진설계적용여부","내진능력"]]

busan_building["시도코드"] = building.apply(make_sido_code, axis=1, meta=object)
busan_building = busan_building[busan_building["시도코드"] == "26"].drop(["시군구코드"], axis=1)

# preprocessing
busan_building["관리건축물대장PK"] = busan_building["관리건축물대장PK"].map(lambda x: x.strip())
busan_building = busan_building.drop("시도코드", axis=1)

# convert dask to pandas
busan_building = busan_building.compute()

# save csv
busan_building.to_csv("data/refined-data/busan-표제부.csv", encoding="utf-8", index=False)

건축물대장 표제부의 분할된 개수는 47개입니다.
Wall time: 5min 32s
